# Create a new Spotify playlist with your Liked Songs 
#### Probably the easiest way to make an OAuth Token is through this link: https://developer.spotify.com/console/post-playlists/

In [64]:
def getInfo():
    token = input("Spotify OAuth Token: ")
    playlist_name = input("Playlist name: ")
    playlist_description = input("Playlist description: ")
    public = False
    if(input("Should it be public? (1-yes, 0-no) ") == '1'):
        public = True
    return token, playlist_name, playlist_description, public

In [65]:
def findNumOfSongs(token):
    endpoint_url = 'https://api.spotify.com/v1/me/tracks'
    query = f'{endpoint_url}'
    response = requests.get(query, headers={"Content-Type":"application/json", "Authorization":f'Bearer {token}'})
    json_response = response.json()
    N = json_response['total']
    print(f'Found {N} Liked Songs.')
    return N

In [66]:
def createPlaylist(token, playlist_name, playlist_description, public):
    endpoint_url = f"https://api.spotify.com/v1/me/playlists?"
    request_body = json.dumps({
              "name": playlist_name,
              "description": playlist_description,
              "public": public
            })
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", "Authorization":f'Bearer {token}'})
    if(response.status_code == 201):
        playlist_id = response.json()['id']
        print(f'Created playlist with id {playlist_id}.')
    else:
        print("Could not create playlist.")
    
    return playlist_id

In [67]:
def scrapeNext50(limit, offset, token):
    market="ES"
    uris = []
    endpoint_url = 'https://api.spotify.com/v1/me/tracks?'
    query = f'{endpoint_url}limit={limit}&market={market}&offset={offset}'
    response = requests.get(query, headers={"Content-Type":"application/json", "Authorization":f'Bearer {token}'})
    if(response.status_code != 200):
        print("Could not scrape songs.")
    json_response = response.json()

    for song in json_response['items']:
        uris.append(song['track']['uri'])
    
    return uris

In [68]:
def addSongs(uris, playlist_id, token):
    endpoint_url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks?'
    request_body = json.dumps({
              "uris" : uris
            })
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", "Authorization":f'Bearer {token}'})

In [69]:
import requests
import json

token, playlist_name, playlist_description, public = getInfo()

N = findNumOfSongs(token)
limit = 50
rem = N%50
max_loop = N-rem

playlist_id = createPlaylist(token, playlist_name, playlist_description, public)

for offset in range(0, max_loop, 50):
    uris = scrapeNext50(limit, offset, token)
    addSongs(uris, playlist_id, token)
uris = scrapeNext50(rem, max_loop, token)
addSongs(uris, playlist_id, token)
print("Songs added to playlist.")